Insalling required libraries 

In [1]:
pip install faker reportlab

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 660.6 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.8 MB 2.1 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.8 MB 4.2 MB/s eta 0:00:01
   ----------------------- ---------------- 1.1/1.8 MB 6.1 MB/s eta 0:00:01
   ------------------------------------- -- 1.7/1.8 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Create a Python Script

In [15]:
import os
from faker import Faker
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Image, Spacer, PageBreak
from reportlab.lib.units import inch, cm

# Initialize Faker
fake = Faker()

# Directories to save PDFs
patient_records_dir = 'patient_records'
invoices_dir = 'invoices'
os.makedirs(patient_records_dir, exist_ok=True)
os.makedirs(invoices_dir, exist_ok=True)

# Clinic/Hospital information
clinic_info = {
    "name": "Sunshine Healthcare",
    "address": "123 Wellness Blvd, Healthy City, HC 45678",
    "phone": "+1 (555) 123-4567",
    "email": "contact@sunshinehealthcare.com",
    "logo": "path_to_logo.png"  # Path to the clinic/hospital logo
}

# Custom styles
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='CenterTitle', alignment=1, fontSize=18, spaceAfter=12, textColor=colors.HexColor('#4B8BBE')))
styles.add(ParagraphStyle(name='NormalCenter', alignment=1, fontSize=12, spaceAfter=6))
styles.add(ParagraphStyle(name='NormalLeft', fontSize=12, spaceAfter=6))
styles.add(ParagraphStyle(name='Disclaimer', alignment=1, fontSize=10, spaceAfter=12, textColor=colors.red))

# Footer content
footer_text = '<a href="http://manishpandey.co.in">manishpandey.co.in</a> | &copy; Manish Pandey'

# Function to add footer
def add_footer(canvas, doc):
    canvas.saveState()
    footer = Paragraph(footer_text, styles['NormalCenter'])
    w, h = footer.wrap(doc.width, doc.bottomMargin)
    footer.drawOn(canvas, doc.leftMargin, h)
    canvas.restoreState()

# Function to generate random health data with selected PHI
def generate_health_data():
    medical_conditions = ['Hypertension', 'Diabetes', 'Asthma', 'Arthritis', 'Heart Disease']
    medications = ['Metformin', 'Lisinopril', 'Albuterol', 'Ibuprofen', 'Atorvastatin']
    return {
        'patient_id': fake.uuid4(),
        'name': fake.name(),
        'dob': fake.date_of_birth(minimum_age=18, maximum_age=90).strftime("%Y-%m-%d"),
        'address': fake.address(),
        'phone': fake.phone_number(),
        'email': fake.email(),
        'health_insurance_number': fake.bothify(text='???-#######'),
        'medicare_beneficiary_number': fake.bothify(text='MBN########'),
        'national_provider_id': fake.bothify(text='NPI#########'),
        'nhs_number': fake.bothify(text='NHS#########'),
        'personal_health_number': fake.bothify(text='PHN#########'),
        'medical_conditions': random.choice(medical_conditions),
        'allergies': fake.word(),
        'medications': random.choice(medications),
        'emergency_contact': fake.name(),
        'emergency_phone': fake.phone_number(),
        'last_visit': fake.date_between(start_date='-1y', end_date='today').strftime("%Y-%m-%d"),
        'doctor': fake.name()
    }

# Function to create a PDF with health data
def create_patient_pdf(patient_data, filename):
    doc = SimpleDocTemplate(filename, pagesize=letter, rightMargin=30, leftMargin=30, topMargin=30, bottomMargin=60)
    elements = []

    disclaimer = Paragraph("Disclaimer: This document contains fake and randomly generated data for demonstration purposes only.", styles['Disclaimer'])
    elements.append(disclaimer)

    if os.path.exists(clinic_info['logo']):
        logo = Image(clinic_info['logo'], 1.5 * inch, 1.5 * inch)
        elements.append(logo)

    elements.append(Spacer(1, 0.5 * cm))

    title = Paragraph(f"{clinic_info['name']}", styles['CenterTitle'])
    elements.append(title)
    address = Paragraph(f"{clinic_info['address']}", styles['NormalCenter'])
    elements.append(address)
    contact = Paragraph(f"Phone: {clinic_info['phone']} | Email: {clinic_info['email']}", styles['NormalCenter'])
    elements.append(contact)

    elements.append(Spacer(1, 1 * cm))

    record_title = Paragraph("Patient Health Record", styles['CenterTitle'])
    elements.append(record_title)

    data = [
        ['Patient ID', patient_data['patient_id']],
        ['Name', patient_data['name']],
        ['DOB', patient_data['dob']],
        ['Address', patient_data['address']],
        ['Phone', patient_data['phone']],
        ['Email', patient_data['email']],
        ['Health Insurance Number', patient_data['health_insurance_number']],
        ['Medicare Beneficiary Number', patient_data['medicare_beneficiary_number']],
        ['National Provider ID', patient_data['national_provider_id']],
        ['NHS Number', patient_data['nhs_number']],
        ['Personal Health Number', patient_data['personal_health_number']],
        ['Medical Conditions', patient_data['medical_conditions']],
        ['Allergies', patient_data['allergies']],
        ['Medications', patient_data['medications']],
        ['Emergency Contact', patient_data['emergency_contact']],
        ['Emergency Phone', patient_data['emergency_phone']],
        ['Last Visit', patient_data['last_visit']],
        ['Doctor', patient_data['doctor']]
    ]

    table = Table(data, colWidths=[2.5 * inch, 3.5 * inch])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#4B8BBE')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#EAF2F8')),
        ('GRID', (0, 0), (-1, -1), 1, colors.HexColor('#D6EAF8')),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('VALIGN', (0, 1), (-1, -1), 'MIDDLE')
    ]))

    elements.append(table)
    elements.append(PageBreak())
    doc.build(elements, onFirstPage=add_footer, onLaterPages=add_footer)

# Function to generate random financial data for an invoice with selected PII
def generate_invoice_data(patient_name):
    services = [
        ("Consultation", round(random.uniform(50, 150), 2)),
        ("Blood Test", round(random.uniform(20, 100), 2)),
        ("X-Ray", round(random.uniform(100, 250), 2)),
        ("MRI", round(random.uniform(500, 1500), 2)),
        ("Medication", round(random.uniform(10, 100), 2)),
        ("Physical Therapy", round(random.uniform(50, 200), 2)),
        ("Vaccination", round(random.uniform(20, 150), 2)),
    ]
    total_amount = sum(service[1] for service in services)
    invoice_date = fake.date_between(start_date='-1y', end_date='today')
    due_date = invoice_date + timedelta(days=30)
    return {
        'invoice_number': fake.bothify(text='INV-######'),
        'patient_name': patient_name,
        'invoice_date': invoice_date.strftime("%Y-%m-%d"),
        'due_date': due_date.strftime("%Y-%m-%d"),
        'services': random.sample(services, k=5),
        'total_amount': round(total_amount, 2),
        'date_of_birth': fake.date_of_birth(minimum_age=18, maximum_age=90).strftime("%Y-%m-%d"),
        'national_identification_number': fake.bothify(text='NIN########'),
        'phone': fake.phone_number(),
        'address': fake.address(),
        'email': fake.email(),
    }

# Function to create a PDF for an invoice
def create_invoice_pdf(invoice_data, filename):
    doc = SimpleDocTemplate(filename, pagesize=letter, rightMargin=30, leftMargin=30, topMargin=30, bottomMargin=60)
    elements = []

    disclaimer = Paragraph("Disclaimer: This document contains fake and randomly generated data for demonstration purposes only.", styles['Disclaimer'])
    elements.append(disclaimer)

    if os.path.exists(clinic_info['logo']):
        logo = Image(clinic_info['logo'], 1.5 * inch, 1.5 * inch)
        elements.append(logo)

    elements.append(Spacer(1, 0.5 * cm))

    title = Paragraph(f"{clinic_info['name']}", styles['CenterTitle'])
    elements.append(title)
    address = Paragraph(f"{clinic_info['address']}", styles['NormalCenter'])
    elements.append(address)
    contact = Paragraph(f"Phone: {clinic_info['phone']} | Email: {clinic_info['email']}", styles['NormalCenter'])
    elements.append(contact)

    elements.append(Spacer(1, 1 * cm))

    invoice_title = Paragraph("Invoice", styles['CenterTitle'])
    elements.append(invoice_title)

    invoice_info = [
        ['Invoice Number', invoice_data['invoice_number']],
        ['Patient Name', invoice_data['patient_name']],
        ['Invoice Date', invoice_data['invoice_date']],
        ['Due Date', invoice_data['due_date']],
        ['Date of Birth', invoice_data['date_of_birth']],
        ['National ID Number', invoice_data['national_identification_number']],
        ['Phone', invoice_data['phone']],
        ['Address', invoice_data['address']],
        ['Email', invoice_data['email']],
    ]

    invoice_table = Table(invoice_info, colWidths=[2.5 * inch, 3.5 * inch])
    invoice_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#4B8BBE')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#EAF2F8')),
        ('GRID', (0, 0), (-1, -1), 1, colors.HexColor('#D6EAF8')),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('VALIGN', (0, 1), (-1, -1), 'MIDDLE')
    ]))

    elements.append(invoice_table)

    elements.append(Spacer(1, 1 * cm))

    services = [["Service", "Cost"]]
    for service, cost in invoice_data['services']:
        services.append([service, f"${cost:.2f}"])

    services_table = Table(services, colWidths=[4 * inch, 2 * inch])
    services_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#4B8BBE')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#EAF2F8')),
        ('GRID', (0, 0), (-1, -1), 1, colors.HexColor('#D6EAF8')),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('VALIGN', (0, 1), (-1, -1), 'MIDDLE')
    ]))

    elements.append(services_table)

    elements.append(Spacer(1, 0.5 * cm))

    total_amount = Paragraph(f"Total Amount: ${invoice_data['total_amount']:.2f}", styles['NormalLeft'])
    elements.append(total_amount)

    elements.append(Spacer(1, 0.5 * cm))

    payment_instructions = Paragraph("Please make payment by the due date. For payment instructions, visit our website or contact us at the phone number above.", styles['NormalLeft'])
    elements.append(payment_instructions)

    elements.append(PageBreak())
    doc.build(elements, onFirstPage=add_footer, onLaterPages=add_footer)

# Generate and save multiple PDF files
num_records = 100  # Number of records to generate
for i in range(num_records):
    # Generate patient data and create patient record PDF
    patient_data = generate_health_data()
    patient_filename = os.path.join(patient_records_dir, f'patient_record_{i+1}.pdf')
    create_patient_pdf(patient_data, patient_filename)
    
    # Generate invoice data and create invoice PDF
    invoice_data = generate_invoice_data(patient_data['name'])
    invoice_filename = os.path.join(invoices_dir, f'invoice_{i+1}.pdf')
    create_invoice_pdf(invoice_data, invoice_filename)

print(f"{num_records} patient records generated in '{patient_records_dir}' directory.")
print(f"{num_records} invoices generated in '{invoices_dir}' directory.")
print(f"Disclaimer: This document contains fake and randomly generated data for demonstration purposes only.")


100 patient records generated in 'patient_records' directory.
100 invoices generated in 'invoices' directory.
Disclaimer: This document contains fake and randomly generated data for demonstration purposes only.
